### 实验目的：
使用一站式在线平台搭建神经网络，实现深度学习算法的网络组建、训练、预测过程
### 实验要求：
完成给定数据集上的分类任务，定义全连接前馈神经网络，softmax输出层以及损失函数 ，训练网路，使用给定的数据集中的测试数据验证精度。
### 基本的实验数据集
MNIST数据集 Cifar10数据集等

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import platform
import tensorflow as tf
print('python_version',platform.python_version())
print('tensorflow',tf.__version__)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

mnist=keras.datasets.mnist

### LossHistory类
保存loss（err）和acc（score）并创建实例,在训练的过程中用实例的来画loss和acc图
### 类中的方法
 1. on_train_begin 方法：初始化 losses，accuracy，val_loss，val_acc
 2. on_batch_end：获取每一次训练的的losses，accuracy，val_loss，val_acc的值
 3. on_epoch_end和on_batch_end作用相同
 4. loss_plot把每一次的losses，accuracy，val_loss，val_acc值的绘画
 
    4.1 画图用的是：matplotlib.pyplot包的 plot方法
        iters x轴，val_acc[loss_type] y轴 ， 'b' 颜色 label 标签的名字
        plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc') 
    

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        #创建一个图
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')#plt.plot(x,y)，这个将数据画成曲线
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)#设置网格形式
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')#给x，y轴加注释
        plt.legend(loc="upper right")#设置图例显示位置
        plt.show()

#创建一个实例LossHistory
history = LossHistory()

 ### 搭建神经网络，识别手写数字
（1）加载数据，引入相关的包

（2）根据网络结构，定义各参数、变量，这是最重要的一步，搭建好网络结构就只需要喂数据

（3）搭建好网络结构，提供数据进行训练，和保存训练模型

（4）用数据训练好模型，

（5）模型评价

In [ ]:
# 获取 数据 
def get_train_val(mnist_path):
    # mnist下载地址：https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data(mnist_path)
    # 打印 训练数据和测试数据的长度
    print("train_images nums:{}".format(len(train_images)))
    print("test_images nums:{}".format(len(test_images)))
    return train_images, train_labels, test_images, test_labels
 
# 展示 图片 数据
def show_mnist(images,labels):
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([ ])
        plt.grid(False)
        plt.imshow(images[i],cmap=plt.cm.gray)
        plt.xlabel(str(labels[i]))
    plt.show()

# one_hot 编码，对标签进行 one_hot
def one_hot(labels):
    onehot_labels=np.zeros(shape=[len(labels),10])
    for i in range(len(labels)):
        index=labels[i]
        onehot_labels[i][index]=1
    return onehot_labels


#### 这里开始搭建网络:一个简单的全连接层网络
1. keras.Sequential() 函数：创建顺序模型
2. model.add函数：来添加网络结构的层数，简单网络一般是输入层，隐含层，输出层三层结构 
3. keras.layers.Dense 函数：创建全连接的网络，有两个关键的参数，一个是神经元的个数units,一个是激活函数的类型activation（注意:每一层神经元的个数是上一层的输出个数）
4. 搭建完输入层，隐含层和输出层，一个简单的神经网络就搭建完成了
5. 最后训练模型
   

In [ ]:
# 搭建 简单 网络 
def mnist_net(input_shape):
    '''
    构建一个简单的全连接层网络模型：
    输入层为28x28=784个输入节点
    隐藏层120个节点
    输出层10个节点
    :param input_shape: 指定输入维度
    :return:
    '''
    # 使用 顺序模型
    model = keras.Sequential()
    # 输入层  input_shape 输入的 大小
    model.add(keras.layers.Flatten(input_shape=input_shape))
    #隐含层  units 神经元的个数120，activation 激活函数 位 rule
    model.add(keras.layers.Dense(units=120, activation=tf.nn.relu)) 
    #输出层 units 神经元的个数10（10个数字），activation 激活函数 位 softmax
    model.add(keras.layers.Dense(units=10, activation=tf.nn.softmax))
    return model

#### CNN网络与前面的相比,多了卷积层和池化层
 1. keras.Sequential() 函数：创建顺序模型
 2. model.add函数：来添加网络结构的层数，简单网络一般是输入层，隐含层，输出层三层结构
 3. keras.layers.Dense函数：创建全连接的网络，有俩个关键的参数，一个是神经元的个数units,一个是激活函数的类型activation 注意:每一层神经元的个数是上一层的输出个数
 4. keras.layers.Conv2D：创建卷积层，过滤器filters,卷积核的大小kernel_size,边缘处理padding,激活函数activation
 5. keras.layers.MaxPool2D：创建完卷积层，然后就是用池化层，这里用最大池化和平均池化，pool_size池化的大小
 6. 最后还是一样，一个全连接的输出层
 7. 训练模型

In [ ]:
# 搭建 CNN 
def mnist_cnn(input_shape):
    '''
    构建一个CNN网络模型
    :param input_shape: 指定输入维度
    :return:
    '''
    # 使用 顺序模型
    model=keras.Sequential()
    # 卷积层 ，kernel_size卷积核大小，activation ，激活函数 ，input_shape 输入大小
    model.add(keras.layers.Conv2D(filters=32,kernel_size = 5,strides = (1,1),
                                  padding = 'same',activation = tf.nn.relu,input_shape = input_shape))
    # 池化层 pool_size 池化大小
    model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides = (2,2), padding = 'valid'))
     # 卷积层 
    model.add(keras.layers.Conv2D(filters=64,kernel_size = 3,strides = (1,1),padding = 'same',activation = tf.nn.relu))
    # 池化层
    model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides = (2,2), padding = 'valid'))
    # 需要丢弃的输入比例。可以防止过拟合
    model.add(keras.layers.Dropout(0.25))
    # 展平一个张量
    model.add(keras.layers.Flatten())
    #全连接层
    model.add(keras.layers.Dense(units=128,activation = tf.nn.relu))
    model.add(keras.layers.Dropout(0.5))
    # 全连接层 输出
    model.add(keras.layers.Dense(units=10,activation = tf.nn.softmax))
    return model


#### 网络结构搭建完后 ，开始训练模型
 1. 在这之前可以对数据做一些处理，正则化等操作，这里用了标准化数据
 2. np.expand_dims 看成增加维度，变化数据的维度
 3. 标签需要one_hot，使用onehot的直接原因是现在多分类cnn网络的输出通常softmax层，而它的输出是一个概率分布，从而要求输入的标签也以概率分布的形式出现，进而算交叉熵之类。
 4. 配置训练模型。optimizer优化器，loss损失函数，学习率，迭代次数等参数
 5. 最后开始训练

In [ ]:
# 训练 模型
def trian_model(train_images,train_labels,test_images,test_labels):
    # re-scale to 0~1.0之间，把数据 压缩到 0-1 之间
    train_images=train_images/255.0
    test_images=test_images/255.0
    # mnist数据转换为四维
    train_images=np.expand_dims(train_images,axis = 3)
    test_images=np.expand_dims(test_images,axis = 3)
    print("train_images :{}".format(train_images.shape))
    print("test_images :{}".format(test_images.shape))
    # 标签 one—hot 
    train_labels=one_hot(train_labels)
    test_labels=one_hot(test_labels)
 
    # 建立模型
    model = mnist_net(input_shape=(28,28,1)) # 简单 网络
    #model=mnist_cnn(input_shape=(28,28,1))  # CNN 网络
    #配置训练模型。optimizer优化器，loss损失函数
    model.compile(optimizer=tf.optimizers.Adam(),loss="categorical_crossentropy",metrics=['accuracy'])
    # 开始 训练 模型 ，callbacks 加入了回调函数，中间获取每次训练的acc和loss 
    model.fit(x=train_images,y=train_labels,epochs=5,validation_data=(test_images,test_labels),callbacks=[history])
    
    # 测试模式 返回 损失值和准确率。
    test_loss,test_acc=model.evaluate(x=test_images,y=test_labels)
    print("Test Accuracy %.2f"%test_acc)
 
    # 开始预测
    cnt=0
    predictions=model.predict(test_images)
    for i in range(len(test_images)):
        target=np.argmax(predictions[i])
        label=np.argmax(test_labels[i])
        if target==label:
            cnt +=1
    # 准确率
    print("True prediction of total : %.2f"%(cnt/len(test_images)))
    # 保存 模型
    model.save('mnist-model.h5')

### 性能评估 

用开始创建的类history绘图的acc和loss变化情况，可以看每一层迭代时的变化情况





**参数对性能的影响:**

可参考 ：https://blog.csdn.net/program_developer/article/details/78597738

这里是举几个参数说明:

激活函数： 

Sigmoid函数：常用的非线性的激活函数，它能够把输入的连续实值变换为0和1之间的输出，特别的，如果是非常大的负数，那么输出就是0；如果是非常大的正数，输出就是1.

tanh函数：解决了Sigmoid函数的不是zero-centered输出问题

Relu函数：Relu目前仍是最常用的activation function，在搭建人工神经网络的时候推荐优先尝试！

迭代次数：epochs（epochs指的就是训练过程中数据将被“轮”多少次）

训练的数量: Batch Size（batch size将决定我们一次训练的样本数目）


In [ ]:
mnist_path = '/kaggle/input/dataet/mnist.npz'
# 获取 数据 
train_images, train_labels, test_images, test_labels=get_train_val(mnist_path)
# 展示 数据 
show_mnist(train_images, train_labels)
# 训练 和 预测
trian_model(train_images, train_labels, test_images, test_labels)
# 绘制acc-loss曲线
history.loss_plot('epoch')

### 要求


参考处理mnist数据集的网络结构，对cifar10数据集进行操作。完成cifar10数据集上的分类任务，定义全连接前馈神经网络，softmax输出层以及损失函数 ，训练网路，使用cifar10的数据集中的测试数据验证精度。

#### 参考操作步骤

（1）加载数据，引入相关的包

（2）根据网络结构，定义各参数、变量，这是最重要的一步，搭建好网络结构就只需要喂数据

（3）搭建好网络结构，提供数据进行训练并保存训练模型

（4）用数据训练好模型

（5）模型评价

#### 搭建网络结构

1. keras.Sequential() 函数创建顺序模型
2. model.add函数来添加网络结构的层数，简单网络一般是输入层，隐含层，输出层三层结构
3. keras.layers.Dense 函数创建全连接的网络，有两个关键的参数，一个是神经元的个数units,一个是激活函数的类型activation，注意:每一层神经元的个数是上一层的输出个数
4. keras.layers.Conv2D 创建卷积层，过滤器filters,卷积核的大小kernel_size,边缘处理padding,激活函数activation
5. keras.layers.MaxPool2D 创建完卷积层，然后就是用池化层，这里用最大池化和平均池化，pool_size池化的大小
6. 最后还是一样，一个全连接的输出层
* 训练模型

#### 网络结构搭建完后 ，开始训练模型
1. 在这之前可以对数据做一些处理，正则化等操作这里用了标准化数据
2. np.expand_dims 看成 增加维度，变化数据的维度
3. 标签需要one_hot，使用onehot的直接原因是现在多分类cnn网络的输出通常是softmax层，而它的输出是一个概率分布，从而要求输入的标签也以概率分布的形式出现，进而算交叉熵之类。
4. 配置训练模型。optimizer优化器，loss损失函数，学习率，迭代次数等参数
* 最后开始训练

#### 性能评估 

用开始创建的类history绘图的acc和loss变化情况，可以看每一层迭代时的变化情况





**参数对性能的影响:**

可参考 ：https://blog.csdn.net/program_developer/article/details/78597738

这里是举几个参数说明:

激活函数： 

Sigmoid函数：常用的非线性的激活函数，它能够把输入的连续实值变换为0和1之间的输出，特别的，如果是非常大的负数，那么输出就是0；如果是非常大的正数，输出就是1.

tanh函数：解决了Sigmoid函数的不是zero-centered输出问题

Relu函数：ReLU目前仍是最常用的activation function，在搭建人工神经网络的时候推荐优先尝试

迭代次数：epochs（epochs指的就是训练过程中数据将被“轮”多少次）

训练的数量: Batch Size（batch size将决定我们一次训练的样本数目）


这里使用ModelCheckpoint 记录中间训练的过程

最后绘制 acc 和 loss 的变化情况